In [ ]:
import pandas as pd
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# récupérer les fichier du dossier contenant les fihcier de clusters
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_brut= glob.glob('/content/drive/MyDrive/Colab Notebooks/km25/*raw*')
list_cluster= glob.glob('/content/drive/MyDrive/Colab Notebooks/km25/*clean*')

In [ ]:
print(list_cluster)

['/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_16_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_6_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_11_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_7_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_2_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_5_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_4_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_15_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_13_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_17_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_1_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_3_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_12_clean_msg.txt', '/content/drive/MyDrive/Colab N

In [ ]:
#afin de correspondre les 2 listes
list_brut.sort()
list_cluster.sort()

In [ ]:
def read_cluster(files):
  liste=[]
  for i,cluster in enumerate(files):
    with open(cluster, "r") as file:
      liste.append(file.readlines())
  return liste

In [ ]:
clean_cluster= read_cluster(list_cluster)

In [ ]:
brut_cluster= read_cluster(list_brut)

In [ ]:
len(clean_cluster)

25

In [ ]:
len(clean_cluster[24])

10955

In [ ]:
for i in range(len(brut_cluster)):
    vars()["data_"+str(i)] = pd.DataFrame(list(zip(brut_cluster[i] ,clean_cluster[i])), columns = ['tweet', 'clean_twt'])

In [ ]:
data_10.head()

,tweet,clean_twt
0,00:85— patience https://t.co/dH7TCxgIeW\n,patience\n
1,00h mon coeur survit pour toi —\n,coeur survit\n
2,02:20 Mon coeur.\n,coeur\n
3,0242 j’aime elise et je ne peux décrire commen...,aime elis pouvoir décrir amour\n
4,0 dignité mais bon https://t.co/g6QuTq9Oyj\n,dignité\n


In [ ]:
!pip install gensim==4.0
!pip install python-Levenshtein

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
w2v = KeyedVectors.load('/content/drive/MyDrive/Colab Notebooks/w2vec_model_d300_1M')

In [ ]:
import numpy as np

In [ ]:
#encode un message
def encode(msg):
    return np.mean([w2v[word] for word in msg if word in w2v] or [np.zeros(300)], axis = 0)

In [ ]:
w2v.similar_by_word("sale")

[('sal', 0.6782066822052002),
 ('pvte', 0.629138708114624),
 ('mèrde', 0.6162537336349487),
 ('pute', 0.6096962094306946),
 ('pte', 0.6044666767120361),
 ('clochard', 0.6014346480369568),
 ('slp', 0.6004031896591187),
 ('crasseux', 0.5957321524620056),
 ('puent', 0.5908592939376831),
 ('daronn', 0.5900101661682129)]

In [ ]:
def encoded(liste):
  w2v=[]
  for i in range(len(liste)):
    w2v.append(np.array([encode(msg.split()) for msg in liste[i]]))
  return w2v

In [ ]:
%%time
#encodage de l'échantillon pour créer le jeu de données
dataset_encoded = encoded(clean_cluster)

CPU times: user 26.5 s, sys: 600 ms, total: 27.1 s
Wall time: 27.3 s


In [ ]:
print(len(dataset_encoded[24]))

10955


In [ ]:
#sauvegarder mes embeddings
%%time
"""for i in range(len(dataset_encoded)):
  with open(f"w2v/w2v{i}.txt", 'a') as f:
      for row in dataset_encoded[i]:
        #np.savetxt(f, row)"""

In [ ]:
from sklearn.ensemble import IsolationForest

clf=IsolationForest(n_estimators=100, max_samples='auto', \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)


In [ ]:
%%time
for i in range(len(dataset_encoded)):
  clf.fit(dataset_encoded[i])

CPU times: user 2min 42s, sys: 1.41 s, total: 2min 43s
Wall time: 1min 27s


In [ ]:
%%time
pred=[]
for i in range(len(dataset_encoded)):
  pred.append(clf.predict(dataset_encoded[i]))

CPU times: user 1min 5s, sys: 434 ms, total: 1min 5s
Wall time: 1min 6s


In [ ]:
print(pred[1])

[1 1 1 ... 1 1 1]


In [ ]:
for i in range(len(dataset_encoded)):
  vars()["data_"+str(i)]['pred']= pred[i]

In [ ]:
data_7.head()

,tweet,clean_twt,pred
0,00:00 rosé\n,roser\n,-1
1,0211OUR CHOICE IS ROSÉ #MAMAVOTE #rosé https:/...,our choice is rosé mamavote roser\n,-1
2,"10,000 votes A.\n",vote\n,-1
3,1000%. VOTE!!!!!!!\n,vot\n,-1
4,1000 votes! Carats❗❗❗❗❗ https://t.co/zEz0uYrAIO\n,vote carats\n,-1


In [ ]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
outliers=[]
for i in range(len(dataset_encoded)):
  outliers.append((vars()["data_"+str(i)]['tweet'].loc[vars()["data_"+str(i)]['pred']==-1]))
  outlier_index=list(outliers[i].index)
  print("cluster"+str(i)+":"+"\n")
  print(vars()["data_"+str(i)]['pred'].value_counts())
  print("\n")

cluster0:

-1    53435
 1     2039
Name: pred, dtype: int64


cluster1:

-1    33627
 1    16255
Name: pred, dtype: int64


cluster2:

-1    1654
 1      16
Name: pred, dtype: int64


cluster3:

-1    55170
 1     5118
Name: pred, dtype: int64


cluster4:

-1    20518
 1     2116
Name: pred, dtype: int64


cluster5:

-1    2674
 1     291
Name: pred, dtype: int64


cluster6:

-1    93164
 1     3550
Name: pred, dtype: int64


cluster7:

-1    2861
 1     326
Name: pred, dtype: int64


cluster8:

-1    2100
 1       1
Name: pred, dtype: int64


cluster9:

-1    19658
 1      489
Name: pred, dtype: int64


cluster10:

-1    20760
 1      683
Name: pred, dtype: int64


cluster11:

-1    48277
 1     1382
Name: pred, dtype: int64


cluster12:

-1    980
 1      4
Name: pred, dtype: int64


cluster13:

-1    7764
 1     252
Name: pred, dtype: int64


cluster14:

-1    3072
 1     150
Name: pred, dtype: int64


cluster15:

-1    30779
 1     7823
Name: pred, dtype: int64


cluster16:

 1    

In [ ]:
for i in range(len(outliers)):
  with open(f"drive/MyDrive/outliers25/outliers{i}", 'w') as f:
    for line in outliers[i]:
      f.write(line)